# Playground for testing random ideas

In [2]:
"""
Non-Zero Registers vs Cardinality in HLLSet
============================================

The relationship follows the "Coupon Collector" problem from probability theory.

Given:
- m = 2^p registers (e.g., 2^10 = 1024 for p_bits=10)
- n = number of distinct elements inserted

Expected number of non-zero (filled) registers:

    E[filled] = m * (1 - (1 - 1/m)^n)
    
For large m, using (1 - 1/m)^n ≈ e^(-n/m):

    E[filled] ≈ m * (1 - e^(-n/m))

Inverse (estimate n from filled registers):

    n ≈ -m * ln(1 - filled/m)

This is related to the "Linear Counting" cardinality estimator!
"""

import numpy as np
import sys
from pathlib import Path

# Setup path
sys.path.insert(0, str(Path('.').absolute()))
from core.hllset import HLLSet, DEFAULT_HASH_CONFIG

def expected_filled_registers(n: int, m: int) -> float:
    """
    Expected number of non-zero registers after inserting n elements.
    
    Uses the coupon collector formula:
        E[filled] = m * (1 - (1 - 1/m)^n)
    """
    if n == 0:
        return 0.0
    # Use log to avoid numerical issues for large n
    # (1 - 1/m)^n = exp(n * log(1 - 1/m))
    log_term = n * np.log1p(-1/m)  # log1p(x) = log(1+x), more accurate for small x
    return m * (1 - np.exp(log_term))


def estimate_cardinality_from_filled(filled: int, m: int) -> float:
    """
    Estimate cardinality from number of filled registers.
    
    This is the "Linear Counting" formula:
        n ≈ -m * ln(1 - filled/m)
    """
    if filled >= m:
        return float('inf')  # All registers filled
    if filled == 0:
        return 0.0
    return -m * np.log(1 - filled/m)


def count_nonzero_registers(hll: HLLSet) -> int:
    """Count non-zero registers in an HLLSet."""
    registers = hll.dump_numpy()
    return np.count_nonzero(registers)


# Test the theory
print("="*60)
print("Non-Zero Registers vs Cardinality Analysis")
print("="*60)

m = 1 << DEFAULT_HASH_CONFIG.p_bits  # 2^p_bits registers
print(f"\nHLLSet configuration:")
print(f"  p_bits = {DEFAULT_HASH_CONFIG.p_bits}")
print(f"  m = 2^{DEFAULT_HASH_CONFIG.p_bits} = {m} registers")

print(f"\n{'n (elements)':<15} {'E[filled]':<12} {'Actual':<10} {'Error %':<10}")
print("-"*50)

test_sizes = [10, 50, 100, 500, 1000, 2000, 5000, 10000]

for n in test_sizes:
    # Create HLLSet from batch of n tokens
    tokens = [f"token_{i}" for i in range(n)]
    hll = HLLSet.from_batch(tokens)
    
    expected = expected_filled_registers(n, m)
    actual = count_nonzero_registers(hll)
    error = 100 * abs(expected - actual) / max(expected, 1)
    
    print(f"{n:<15} {expected:<12.1f} {actual:<10} {error:<10.2f}")

Non-Zero Registers vs Cardinality Analysis

HLLSet configuration:
  p_bits = 10
  m = 2^10 = 1024 registers

n (elements)    E[filled]    Actual     Error %   
--------------------------------------------------
10              10.0         10         0.44      
50              48.8         49         0.36      
100             95.3         94         1.38      
500             395.7        400        1.08      
1000            638.5        627        1.81      
2000            878.9        885        0.69      
5000            1016.3       1020       0.37      
10000           1023.9       1024       0.01      


In [3]:
# Inverse: Estimate cardinality from filled registers
print("\n" + "="*60)
print("Inverse: Estimating Cardinality from Filled Registers")
print("="*60)
print("\nUsing Linear Counting formula: n ≈ -m * ln(1 - filled/m)")

print(f"\n{'Actual n':<12} {'Filled':<10} {'Est. n':<12} {'HLL Est.':<12} {'LC Error%':<10}")
print("-"*60)

for n in test_sizes:
    tokens = [f"token_{i}" for i in range(n)]
    hll = HLLSet.from_batch(tokens)
    
    filled = count_nonzero_registers(hll)
    est_from_filled = estimate_cardinality_from_filled(filled, m)
    hll_estimate = hll.cardinality()  # HLL's own estimate
    
    lc_error = 100 * abs(est_from_filled - n) / n
    
    print(f"{n:<12} {filled:<10} {est_from_filled:<12.1f} {hll_estimate:<12.1f} {lc_error:<10.2f}")

# Key insight
print("\n" + "="*60)
print("KEY FORMULAS")
print("="*60)
print("""
Given m = 2^p registers:

1. FILLED FROM CARDINALITY (Coupon Collector):
   
   filled(n) = m × (1 - e^(-n/m))
   
2. CARDINALITY FROM FILLED (Linear Counting):
   
   n(filled) = -m × ln(1 - filled/m)

3. SATURATION POINT:
   - At n = m: ~63.2% of registers filled
   - At n = 3m: ~95% of registers filled
   - At n = 5m: ~99.3% of registers filled

4. PRACTICAL LIMITS:
   - For accurate Linear Counting: filled < 0.7m
   - When filled ≈ m: switch to HLL estimator
""")

# Demonstrate saturation
print("\n" + "="*60)
print("Saturation Analysis")
print("="*60)
print(f"\nm = {m} registers")
print(f"\n{'n/m ratio':<12} {'n':<10} {'E[filled]':<12} {'% filled':<10}")
print("-"*45)

for ratio in [0.1, 0.5, 1.0, 2.0, 3.0, 5.0, 10.0]:
    n = int(m * ratio)
    expected = expected_filled_registers(n, m)
    pct = 100 * expected / m
    print(f"{ratio:<12} {n:<10} {expected:<12.1f} {pct:<10.1f}%")


Inverse: Estimating Cardinality from Filled Registers

Using Linear Counting formula: n ≈ -m * ln(1 - filled/m)

Actual n     Filled     Est. n       HLL Est.     LC Error% 
------------------------------------------------------------
10           10         10.0         12.0         0.49      
50           49         50.2         55.0         0.42      
100          94         98.6         106.0        1.40      
500          400        507.2        514.0        1.44      
1000         627        970.3        995.0        2.97      
2000         885        2044.9       2126.0       2.25      
5000         1020       5678.3       5442.0       13.57     
10000        1024       inf          10211.0      inf       

KEY FORMULAS

Given m = 2^p registers:

1. FILLED FROM CARDINALITY (Coupon Collector):

   filled(n) = m × (1 - e^(-n/m))

2. CARDINALITY FROM FILLED (Linear Counting):

   n(filled) = -m × ln(1 - filled/m)

3. SATURATION POINT:
   - At n = m: ~63.2% of registers filled
   -

In [5]:
"""
Non-Zero BITS in HLLSet Registers
=================================

Each register stores max(ρ) where ρ = trailing_zeros + 1.
Register values range from 0 to ~31 (for 64-bit hash with p_bits=10).

Two metrics:
1. Total 1-bits (popcount): sum of popcount(register_i) for all i
2. Total bit-width: sum of bit_length(register_i) for all i

Theory:
- Expected max ρ for a register with k elements: E[max] ≈ log₂(k) + γ/ln(2)
  where γ ≈ 0.5772 (Euler's constant)
- For n total elements, k ≈ n/m elements per register (on average)
"""

def count_total_bits(hll: HLLSet) -> dict:
    """Count various bit metrics in HLLSet registers."""
    registers = hll.dump_numpy()
    
    # Total 1-bits (popcount across all registers)
    total_ones = sum(bin(int(r)).count('1') for r in registers)
    
    # Total bit-width (bits needed to represent each value)
    total_bitwidth = sum(int(r).bit_length() for r in registers)
    
    # Sum of register values
    total_value = int(registers.sum())
    
    # Max register value
    max_value = int(registers.max())
    
    # Non-zero registers
    nonzero_regs = np.count_nonzero(registers)
    
    return {
        'total_ones': total_ones,        # Sum of popcount
        'total_bitwidth': total_bitwidth, # Sum of bit_length
        'total_value': total_value,       # Sum of values
        'max_value': max_value,           # Max single value
        'nonzero_regs': nonzero_regs,
        'avg_value': total_value / max(nonzero_regs, 1),
    }


def expected_register_value(k: float) -> float:
    """
    Expected max trailing zeros for a register with k elements.
    
    For k iid draws from geometric(1/2), the expected max is:
        E[max] ≈ log₂(k) + γ/ln(2) ≈ log₂(k) + 0.8327
    """
    if k <= 0:
        return 0.0
    gamma = 0.5772156649  # Euler-Mascheroni constant
    return np.log2(k) + gamma / np.log(2)


def expected_total_bits(n: int, m: int) -> dict:
    """
    Estimate total bits given n elements and m registers.
    """
    if n == 0:
        return {'total_ones': 0, 'total_bitwidth': 0, 'total_value': 0}
    
    # Expected elements per register
    k_avg = n / m
    
    # Expected filled registers
    filled = m * (1 - np.exp(-n/m))
    
    # Expected value per filled register
    avg_value = expected_register_value(k_avg)
    
    # Total value across all registers
    total_value = filled * avg_value
    
    # Approximate bit-width: for value v, bit_length = floor(log2(v)) + 1
    avg_bitwidth = max(1, int(np.log2(max(avg_value, 1)))) + 1
    total_bitwidth = filled * avg_bitwidth
    
    # 1-bits: approximately half of the bit positions are 1
    total_ones = total_bitwidth * 0.5
    
    return {
        'total_ones': total_ones,
        'total_bitwidth': total_bitwidth,
        'total_value': total_value,
        'avg_value': avg_value,
        'filled': filled,
    }


# Test bit analysis
print("="*70)
print("Non-Zero BITS in HLLSet Registers")
print("="*70)

print(f"\nm = {m} registers, max register value ≈ 31 (5 bits each)")
print(f"Theoretical max total bits = {m} × 5 = {m * 5}")

print(f"\n{'n':<10} {'Filled':<8} {'Σvalues':<10} {'Σ1-bits':<10} {'Σbitwidth':<10} {'avg_val':<8} {'max_val':<8}")
print("-"*70)

for n in test_sizes:
    tokens = [f"token_{i}" for i in range(n)]
    hll = HLLSet.from_batch(tokens)
    
    bits = count_total_bits(hll)
    
    print(f"{n:<10} {bits['nonzero_regs']:<8} {bits['total_value']:<10} "
          f"{bits['total_ones']:<10} {bits['total_bitwidth']:<10} "
          f"{bits['avg_value']:<8.2f} {bits['max_value']:<8}")

Non-Zero BITS in HLLSet Registers

m = 1024 registers, max register value ≈ 31 (5 bits each)
Theoretical max total bits = 1024 × 5 = 5120

n          Filled   Σvalues    Σ1-bits    Σbitwidth  avg_val  max_val 
----------------------------------------------------------------------
10         10       119        10         30         11.90    64      
50         49       435        50         124        8.88     128     
100        94       557        99         215        5.93     128     
500        400      2500       467        878        6.25     516     
1000       627      5280       862        1479       8.42     1024    
2000       885      12149      1519       2446       13.73    2051    
5000       1020     35761      2730       3788       35.06    4119    
10000      1024     72351      3723       4738       70.66    16399   


In [6]:
# Compare actual vs expected
print("\n" + "="*70)
print("Comparison: Actual vs Expected (Theory)")
print("="*70)

print(f"\n{'n':<10} {'Act.Σval':<12} {'Exp.Σval':<12} {'Act.bits':<10} {'Exp.bits':<10} {'Err%':<8}")
print("-"*65)

for n in test_sizes:
    tokens = [f"token_{i}" for i in range(n)]
    hll = HLLSet.from_batch(tokens)
    
    actual = count_total_bits(hll)
    expected = expected_total_bits(n, m)
    
    val_err = 100 * abs(actual['total_value'] - expected['total_value']) / max(expected['total_value'], 1)
    
    print(f"{n:<10} {actual['total_value']:<12} {expected['total_value']:<12.0f} "
          f"{actual['total_ones']:<10} {expected['total_ones']:<10.0f} {val_err:<8.1f}")

# Key formulas
print("\n" + "="*70)
print("KEY FORMULAS FOR NON-ZERO BITS")
print("="*70)
print("""
Given m = 2^p registers, n elements:

1. EXPECTED REGISTER VALUE (Extreme Value Theory):
   
   For k elements in a register:
   E[max(ρ₁, ρ₂, ..., ρₖ)] ≈ log₂(k) + γ/ln(2) ≈ log₂(k) + 0.833
   
   where γ = 0.5772 (Euler's constant)

2. TOTAL REGISTER VALUES:
   
   Σvalues ≈ filled × E[max(ρ)] 
          ≈ m(1 - e^(-n/m)) × (log₂(n/m) + 0.833)

3. TOTAL 1-BITS (popcount):
   
   Σ1-bits ≈ Σbitwidth × 0.5
           ≈ filled × (log₂(avg_value) + 1) × 0.5

4. SCALING BEHAVIOR:
   - Small n (n << m): Linear with n
   - Large n (n >> m): Logarithmic with n (registers saturate)
   
5. MAXIMUM POSSIBLE:
   - Max register value: ~31 (5 bits for 64-bit hash)
   - Max total bits: m × 5 = {0}
   - Max 1-bits: m × 5 × 0.5 ≈ {1}
""".format(m * 5, m * 5 // 2))

# Show saturation behavior
print("="*70)
print("Bit Saturation Analysis")  
print("="*70)

print(f"\n{'n/m':<10} {'n':<10} {'Σvalues':<12} {'Σ1-bits':<10} {'bits/reg':<10}")
print("-"*55)

for ratio in [0.1, 0.5, 1, 2, 5, 10, 50, 100]:
    n = int(m * ratio)
    tokens = [f"token_{i}" for i in range(n)]
    hll = HLLSet.from_batch(tokens)
    bits = count_total_bits(hll)
    
    bits_per_reg = bits['total_ones'] / m
    print(f"{ratio:<10} {n:<10} {bits['total_value']:<12} {bits['total_ones']:<10} {bits_per_reg:<10.2f}")


Comparison: Actual vs Expected (Theory)

n          Act.Σval     Exp.Σval     Act.bits   Exp.bits   Err%    
-----------------------------------------------------------------
10         119          -58          10         10         17716.9 
50         435          -172         50         49         60693.8 
100        557          -240         99         95         79741.0 
500        2500         -80          467        396        257970.0
1000       5280         510          862        638        935.8   
2000       12149        1580         1519       879        668.7   
5000       35761        3171         2730       1016       1027.7  
10000      72351        4219         3723       1536       1614.8  

KEY FORMULAS FOR NON-ZERO BITS

Given m = 2^p registers, n elements:

1. EXPECTED REGISTER VALUE (Extreme Value Theory):

   For k elements in a register:
   E[max(ρ₁, ρ₂, ..., ρₖ)] ≈ log₂(k) + γ/ln(2) ≈ log₂(k) + 0.833

   where γ = 0.5772 (Euler's constant)

2. TOTAL REGISTER 

# Entanglement as Graph Isomorphism

## The Discovery

Given the **same dataset** processed through HLLSets with **different seeds**:
- Group 1: `HLLSet(data, seed=42)` → Graph G₁
- Group 2: `HLLSet(data, seed=137)` → Graph G₂

**Key observation**: G₁ and G₂ are **isomorphic** despite having completely different bit representations!

## Node Tuple Fingerprint

Each node is uniquely characterized by:
```
node = (cardinality, in_edges[(τ₁,ρ₁), (τ₂,ρ₂), ...], out_edges[(τ₁,ρ₁), (τ₂,ρ₂), ...])
```

Where:
- `cardinality` = |HLLSet|
- `τ` (tau) = **BSS similarity** = intersection(A,B) / cardinality(A)
- `ρ` (rho) = **BSS dissimilarity** = cardinality({aᵢ ≠ bᵢ}) / cardinality(A)

**Note**: τ and ρ together provide **two-sided bounds** on similarity between HLLSets A and B (Bell State Similarity).

**Perfect entanglement**: Matching nodes have identical tuples.
**Approximate entanglement**: Matching nodes have "close enough" tuples.

## Combinatorial Bound

Total information capacity: **2^p × 32 bits** (e.g., 1024 × 32 = 32,768 bits)

This bounds the number of distinguishable states, making the graph matching tractable!

## NitroSAT for Graph Isomorphism

The node mapping problem is naturally a SAT problem:
- Variable `x_{i,j} = 1` means node i in G₁ maps to node j in G₂
- Constraints ensure valid bijection
- Soft constraints for tuple similarity

In [7]:
"""
Entanglement Graph Matching via NitroSAT
=========================================

Given two graphs G₁, G₂ derived from entangled HLLSets,
find the node mapping that demonstrates isomorphism.
"""

from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Set, Optional
from collections import defaultdict

@dataclass(frozen=True)
class NodeTuple:
    """
    Node fingerprint for entanglement detection.
    
    A node is characterized by:
    - cardinality: |HLLSet| of this node
    - in_edges: list of (edge_type, weight) for incoming edges
    - out_edges: list of (edge_type, weight) for outgoing edges
    """
    cardinality: int
    in_degree: int
    out_degree: int
    # Sorted edge signatures for comparison
    in_signature: Tuple[Tuple[str, int], ...] = ()
    out_signature: Tuple[Tuple[str, int], ...] = ()
    
    def distance(self, other: 'NodeTuple') -> float:
        """Compute distance between node tuples."""
        if self.in_degree != other.in_degree or self.out_degree != other.out_degree:
            return float('inf')  # Degree mismatch = not compatible
        
        # Cardinality difference (normalized)
        card_diff = abs(self.cardinality - other.cardinality) / max(self.cardinality, other.cardinality, 1)
        
        # Edge signature similarity (Jaccard-like)
        # For now, simple comparison
        return card_diff
    
    def is_compatible(self, other: 'NodeTuple', threshold: float = 0.1) -> bool:
        """Check if two nodes could be entangled partners."""
        return self.distance(other) < threshold


@dataclass
class EntanglementGraph:
    """
    Graph representation for entanglement analysis.
    
    Nodes are identified by ID, with associated NodeTuple fingerprints.
    """
    name: str
    nodes: Dict[int, NodeTuple] = field(default_factory=dict)
    edges: List[Tuple[int, int, str, int]] = field(default_factory=list)  # (src, dst, type, weight)
    
    def add_node(self, node_id: int, cardinality: int, 
                 in_edges: List[Tuple[str, int]] = None,
                 out_edges: List[Tuple[str, int]] = None):
        """Add a node with its fingerprint."""
        in_edges = in_edges or []
        out_edges = out_edges or []
        
        self.nodes[node_id] = NodeTuple(
            cardinality=cardinality,
            in_degree=len(in_edges),
            out_degree=len(out_edges),
            in_signature=tuple(sorted(in_edges)),
            out_signature=tuple(sorted(out_edges)),
        )
    
    def add_edge(self, src: int, dst: int, edge_type: str = "default", weight: int = 1):
        """Add an edge."""
        self.edges.append((src, dst, edge_type, weight))
    
    def degree_sequence(self) -> List[Tuple[int, int]]:
        """Get sorted degree sequence (in, out) for graph fingerprint."""
        return sorted((n.in_degree, n.out_degree) for n in self.nodes.values())
    
    def cardinality_sequence(self) -> List[int]:
        """Get sorted cardinality sequence."""
        return sorted(n.cardinality for n in self.nodes.values())


def find_compatible_pairs(g1: EntanglementGraph, g2: EntanglementGraph, 
                          threshold: float = 0.1) -> Dict[int, List[int]]:
    """
    Find potentially compatible node pairs between two graphs.
    
    Returns dict mapping g1_node_id -> list of compatible g2_node_ids.
    """
    compatible = defaultdict(list)
    
    for id1, node1 in g1.nodes.items():
        for id2, node2 in g2.nodes.items():
            if node1.is_compatible(node2, threshold):
                compatible[id1].append(id2)
    
    return dict(compatible)


def entanglement_to_sat(g1: EntanglementGraph, g2: EntanglementGraph,
                        compatible_pairs: Dict[int, List[int]]) -> Tuple[int, List[List[int]], Dict]:
    """
    Convert entanglement mapping problem to SAT.
    
    Variables: x_{i,j} = 1 means node i in G₁ maps to node j in G₂
    
    Constraints:
    1. Each node in G₁ maps to exactly one node in G₂
    2. Each node in G₂ is mapped from at most one node in G₁
    3. Only compatible pairs can be mapped
    
    Returns: (num_vars, clauses, var_mapping)
    """
    # Create variable mapping
    var_map = {}  # (id1, id2) -> var_num
    reverse_map = {}  # var_num -> (id1, id2)
    var_counter = 1
    
    for id1, compatible_ids in compatible_pairs.items():
        for id2 in compatible_ids:
            var_map[(id1, id2)] = var_counter
            reverse_map[var_counter] = (id1, id2)
            var_counter += 1
    
    num_vars = var_counter - 1
    clauses = []
    
    # Constraint 1: Each node in G₁ maps to exactly one node in G₂
    for id1 in g1.nodes:
        if id1 not in compatible_pairs:
            continue  # No compatible mapping - problem may be UNSAT
        
        # At-least-one: (x_{i,j1} OR x_{i,j2} OR ...)
        vars_for_node = [var_map[(id1, id2)] for id2 in compatible_pairs[id1]]
        if vars_for_node:
            clauses.append(vars_for_node)
        
        # At-most-one: for each pair, NOT both
        for k1 in range(len(vars_for_node)):
            for k2 in range(k1 + 1, len(vars_for_node)):
                clauses.append([-vars_for_node[k1], -vars_for_node[k2]])
    
    # Constraint 2: Each node in G₂ is mapped from at most one node in G₁
    g2_to_vars = defaultdict(list)
    for (id1, id2), var in var_map.items():
        g2_to_vars[id2].append(var)
    
    for id2, vars_for_node in g2_to_vars.items():
        # At-most-one
        for k1 in range(len(vars_for_node)):
            for k2 in range(k1 + 1, len(vars_for_node)):
                clauses.append([-vars_for_node[k1], -vars_for_node[k2]])
    
    return num_vars, clauses, reverse_map


# Demo: Create two entangled graphs
print("="*70)
print("Entanglement Graph Isomorphism Demo")
print("="*70)

# Graph 1: Original representation (seed=42)
g1 = EntanglementGraph(name="G1_seed42")
g1.add_node(0, cardinality=1000, in_edges=[], out_edges=[("link", 2)])
g1.add_node(1, cardinality=500, in_edges=[("link", 1)], out_edges=[("link", 1)])
g1.add_node(2, cardinality=750, in_edges=[("link", 1)], out_edges=[])

# Graph 2: Entangled representation (seed=137) - same structure, different node IDs
g2 = EntanglementGraph(name="G2_seed137")
g2.add_node(10, cardinality=1005, in_edges=[], out_edges=[("link", 2)])  # Maps to 0
g2.add_node(11, cardinality=498, in_edges=[("link", 1)], out_edges=[("link", 1)])  # Maps to 1
g2.add_node(12, cardinality=752, in_edges=[("link", 1)], out_edges=[])  # Maps to 2

print(f"\n{g1.name}:")
for nid, node in g1.nodes.items():
    print(f"  Node {nid}: card={node.cardinality}, in={node.in_degree}, out={node.out_degree}")

print(f"\n{g2.name}:")
for nid, node in g2.nodes.items():
    print(f"  Node {nid}: card={node.cardinality}, in={node.in_degree}, out={node.out_degree}")

# Find compatible pairs
compatible = find_compatible_pairs(g1, g2, threshold=0.1)
print(f"\nCompatible pairs (threshold=0.1):")
for id1, id2_list in compatible.items():
    print(f"  G1[{id1}] -> G2{id2_list}")

# Convert to SAT
num_vars, clauses, var_mapping = entanglement_to_sat(g1, g2, compatible)
print(f"\nSAT formulation:")
print(f"  Variables: {num_vars}")
print(f"  Clauses: {len(clauses)}")

Entanglement Graph Isomorphism Demo

G1_seed42:
  Node 0: card=1000, in=0, out=1
  Node 1: card=500, in=1, out=1
  Node 2: card=750, in=1, out=0

G2_seed137:
  Node 10: card=1005, in=0, out=1
  Node 11: card=498, in=1, out=1
  Node 12: card=752, in=1, out=0

Compatible pairs (threshold=0.1):
  G1[0] -> G2[10]
  G1[1] -> G2[11]
  G1[2] -> G2[12]

SAT formulation:
  Variables: 3
  Clauses: 3


In [8]:
# Solve with NitroSAT
from nitrosat import CNFFormula, NitroSatSolver

def solve_entanglement_mapping(g1: EntanglementGraph, g2: EntanglementGraph,
                                threshold: float = 0.1) -> Optional[Dict[int, int]]:
    """
    Find the entanglement mapping between two graphs using NitroSAT.
    
    Returns: Dict mapping g1_node_id -> g2_node_id, or None if no mapping exists.
    """
    # Find compatible pairs
    compatible = find_compatible_pairs(g1, g2, threshold)
    
    if not compatible:
        return None
    
    # Convert to SAT
    num_vars, clauses, var_mapping = entanglement_to_sat(g1, g2, compatible)
    
    if num_vars == 0:
        return None
    
    # Build CNF formula
    formula = CNFFormula(num_vars=num_vars)
    for clause in clauses:
        formula.add_clause(*clause)
    
    # Solve
    solver = NitroSatSolver(verbose=False)
    result = solver.solve(formula)
    
    if not result.solved:
        return None
    
    # Extract mapping
    mapping = {}
    for var, (id1, id2) in var_mapping.items():
        if result.get_assignment(var) == 1:
            mapping[id1] = id2
    
    return mapping


# Solve the demo
print("="*70)
print("Solving Entanglement Mapping with NitroSAT")
print("="*70)

mapping = solve_entanglement_mapping(g1, g2, threshold=0.1)

if mapping:
    print(f"\n✓ Entanglement mapping found!")
    print(f"\n  G1 Node  →  G2 Node")
    print(f"  " + "-"*20)
    for id1, id2 in sorted(mapping.items()):
        node1 = g1.nodes[id1]
        node2 = g2.nodes[id2]
        print(f"  {id1:>6}  →  {id2:>6}  (card: {node1.cardinality} ↔ {node2.cardinality})")
else:
    print("\n✗ No valid entanglement mapping exists")

# Now test with a harder case - more nodes, some ambiguity
print("\n" + "="*70)
print("Harder Case: Multiple Compatible Candidates")
print("="*70)

# Create graphs with some structural ambiguity
g3 = EntanglementGraph(name="G3")
g4 = EntanglementGraph(name="G4")

# G3: 5 nodes with some similar fingerprints
g3.add_node(0, cardinality=1000, in_edges=[], out_edges=[("link", 2)])
g3.add_node(1, cardinality=500, in_edges=[("link", 1)], out_edges=[("link", 1)])
g3.add_node(2, cardinality=502, in_edges=[("link", 1)], out_edges=[("link", 1)])  # Similar to 1!
g3.add_node(3, cardinality=750, in_edges=[("link", 1)], out_edges=[])
g3.add_node(4, cardinality=748, in_edges=[("link", 1)], out_edges=[])  # Similar to 3!

# G4: Same structure, shuffled IDs
g4.add_node(20, cardinality=1002, in_edges=[], out_edges=[("link", 2)])
g4.add_node(21, cardinality=501, in_edges=[("link", 1)], out_edges=[("link", 1)])
g4.add_node(22, cardinality=499, in_edges=[("link", 1)], out_edges=[("link", 1)])
g4.add_node(23, cardinality=751, in_edges=[("link", 1)], out_edges=[])
g4.add_node(24, cardinality=749, in_edges=[("link", 1)], out_edges=[])

compatible34 = find_compatible_pairs(g3, g4, threshold=0.1)
print(f"\nCompatible pairs:")
for id1, id2_list in sorted(compatible34.items()):
    print(f"  G3[{id1}] -> G4{id2_list}")

mapping34 = solve_entanglement_mapping(g3, g4, threshold=0.1)

if mapping34:
    print(f"\n✓ Mapping found with {len(mapping34)} node pairs")
    for id1, id2 in sorted(mapping34.items()):
        print(f"  {id1} → {id2}")
else:
    print("\n✗ No valid mapping")

Solving Entanglement Mapping with NitroSAT

✓ Entanglement mapping found!

  G1 Node  →  G2 Node
  --------------------
       0  →      10  (card: 1000 ↔ 1005)
       1  →      11  (card: 500 ↔ 498)
       2  →      12  (card: 750 ↔ 752)

Harder Case: Multiple Compatible Candidates

Compatible pairs:
  G3[0] -> G4[20]
  G3[1] -> G4[21, 22]
  G3[2] -> G4[21, 22]
  G3[3] -> G4[23, 24]
  G3[4] -> G4[23, 24]

✓ Mapping found with 5 node pairs
  0 → 20
  1 → 21
  2 → 22
  3 → 23
  4 → 24


In [9]:
# Complexity analysis: How bits bound the graph matching problem
print("\n" + "="*70)
print("Bit Capacity and Combinatorial Bounds")
print("="*70)

p_bits = DEFAULT_HASH_CONFIG.p_bits
m = 1 << p_bits
bits_per_register = 32  # Using uint32

total_bits = m * bits_per_register
print(f"""
HLLSet Bit Capacity:
  Registers: m = 2^{p_bits} = {m}
  Bits per register: {bits_per_register}
  Total bits: {total_bits:,} ({total_bits // 8:,} bytes)

Theoretical bounds on distinguishable states:
  Maximum HLLSets: 2^{total_bits} (astronomical)
  
  But practical limits are much smaller:
  - Register values range [0, ~31] → ~5 bits effective per register
  - Effective bits: ~{m * 5:,}
  - Non-zero registers bounded by cardinality (Coupon Collector)
""")

# How does this affect graph matching?
print("="*70)
print("Graph Matching Complexity with HLLSet Nodes")
print("="*70)

def estimate_matching_complexity(num_nodes: int, avg_compatible: float) -> dict:
    """
    Estimate SAT complexity for graph matching.
    
    Worst case (no fingerprint filtering): n! permutations
    With fingerprints: much smaller search space
    """
    import math
    
    # Without filtering
    brute_force = math.factorial(num_nodes) if num_nodes <= 20 else float('inf')
    
    # With fingerprint filtering (avg_compatible candidates per node)
    filtered = avg_compatible ** num_nodes
    
    # SAT formulation size
    sat_vars = int(num_nodes * avg_compatible)
    # At-least-one: n clauses of size avg_compatible
    # At-most-one: n * C(avg_compatible, 2) clauses
    sat_clauses = int(num_nodes + num_nodes * avg_compatible * (avg_compatible - 1) / 2)
    
    return {
        'brute_force': brute_force,
        'filtered_search': filtered,
        'sat_variables': sat_vars,
        'sat_clauses': sat_clauses,
        'complexity_reduction': brute_force / max(filtered, 1) if brute_force < float('inf') else float('inf'),
    }

print(f"\n{'Nodes':<8} {'Brute Force':<15} {'Filtered':<15} {'SAT Vars':<10} {'Reduction':<12}")
print("-"*60)

for num_nodes in [5, 10, 20, 50, 100, 1000]:
    # Assume fingerprints reduce to ~2 compatible candidates per node on average
    stats = estimate_matching_complexity(num_nodes, avg_compatible=2.0)
    
    bf = stats['brute_force']
    bf_str = f"{bf:.2e}" if bf < float('inf') else "∞"
    
    print(f"{num_nodes:<8} {bf_str:<15} {stats['filtered_search']:<15.0f} "
          f"{stats['sat_variables']:<10} {stats['complexity_reduction']:.2e}")

print("""
KEY INSIGHT:
===========
The NodeTuple fingerprint (cardinality, in_degree, out_degree, edge_signatures)
acts as a HASH that partitions the search space.

Good fingerprints → few compatible candidates per node → tractable SAT
Poor fingerprints → many candidates → harder SAT (but still polynomial!)

NitroSAT's physics-inspired approach (spectral geometry, heat kernel, BAHA)
exploits the structure of the compatibility graph to find solutions efficiently.
""")


Bit Capacity and Combinatorial Bounds

HLLSet Bit Capacity:
  Registers: m = 2^10 = 1024
  Bits per register: 32
  Total bits: 32,768 (4,096 bytes)

Theoretical bounds on distinguishable states:
  Maximum HLLSets: 2^32768 (astronomical)

  But practical limits are much smaller:
  - Register values range [0, ~31] → ~5 bits effective per register
  - Effective bits: ~5,120
  - Non-zero registers bounded by cardinality (Coupon Collector)

Graph Matching Complexity with HLLSet Nodes

Nodes    Brute Force     Filtered        SAT Vars   Reduction   
------------------------------------------------------------
5        1.20e+02        32              10         3.75e+00
10       3.63e+06        1024            20         3.54e+03
20       2.43e+18        1048576         40         2.32e+12
50       ∞               1125899906842624 100        inf
100      ∞               1267650600228229401496703205376 200        inf
1000     ∞               107150860718626732094842504906000181056140481170553